In [12]:
import pandas as pd
import re
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

# ==============================================================
# 1. Load Data & Define Standardization Functions
# ==============================================================
df = pd.read_excel('C:/Users/user/OneDrive/gym_recommendation_system/venv/data/gym recommendation.xlsx')

def standardize_text(text):
    """Clean and normalize text entries"""
    text = re.sub(r'[^a-zA-Z0-9 ]', '', str(text))  # Remove special chars
    text = text.strip().lower()
    # Manual term replacements
    replacements = {
        'dumbbells': 'dumbbell',
        'treadmills': 'treadmill',
        'swimming.': 'swimming',
        'carrots': 'carrot',
        'male': 0, 'female': 1,  # Predefine gender encoding
        'yes': 1, 'no': 0        # Predefine binary flags
    }
    return replacements.get(text, text)

# =================================================================
# 2. Process Multi-Label Columns
# =================================================================
def process_diet(diet_str):
    """Process diet column with nested structure"""
    diet_str = re.sub(r'\w+:\s*', '', str(diet_str)) 
    items = re.findall(r'\((.*?)\)', diet_str)
    return [standardize_text(item) 
            for group in items 
            for item in re.split(r', | and ', group) 
            if item.strip()]

def process_generic(column_str):
    """Process Exercises/Equipment columns"""
    return [standardize_text(item) 
            for item in re.split(r', | and ', str(column_str)) 
            if item.strip()]

# Apply processing
df['Diet'] = df['Diet'].apply(process_diet)
df['Exercises'] = df['Exercises'].apply(process_generic)
df['Equipment'] = df['Equipment'].apply(process_generic)

# =================================================================
# 3. Encode Multi-Label Features
# =================================================================
class DedupeMLB(MultiLabelBinarizer):
    """Custom MLB with text standardization"""
    def fit_transform(self, y):
        y_clean = [[standardize_text(item) for item in lst] for lst in y]
        return super().fit_transform(y_clean)

def safe_encode(df, column):
    """Encoding with automatic deduplication"""
    mlb = DedupeMLB()
    encoded = pd.DataFrame(
        mlb.fit_transform(df[column]),
        columns=[f"{column}_{item.replace(' ', '_')}" for item in mlb.classes_],
        index=df.index
    )
    return encoded.T.groupby(level=0).max().T

# Encode all multi-label columns
diet_encoded = safe_encode(df, 'Diet')
exercises_encoded = safe_encode(df, 'Exercises')
equipment_encoded = safe_encode(df, 'Equipment')

# =================================================================
# 4. Encode Categorical Variables
# =================================================================
# Convert binary columns
binary_cols = ['Sex', 'Hypertension', 'Diabetes']
df[binary_cols] = df[binary_cols].applymap(standardize_text)

# One-hot encode multi-class columns
cat_cols = ['Level', 'Fitness Goal', 'Fitness Type']
ohe = OneHotEncoder(sparse_output=False, drop='first')
ohe_features = ohe.fit_transform(df[cat_cols])
ohe_df = pd.DataFrame(ohe_features, columns=ohe.get_feature_names_out(cat_cols))

# =================================================================
# 5. Create Final DataFrame
# =================================================================
final_df = pd.concat([
    df.drop(['Diet', 'Exercises', 'Equipment', 'ID', 'Recommendation'] + cat_cols, axis=1),
    ohe_df,
    diet_encoded.add_prefix('diet_'),
    exercises_encoded.add_prefix('exercise_'),
    equipment_encoded.add_prefix('equipment_')
], axis=1).astype(float)

# =================================================================
# 6. Model Training (Revised)
# =================================================================
# Verify columns
print("Final features:", final_df.columns.tolist()[:30], "...")

# Isolate key features
feature_cols = ['Age', 'BMI', 'Hypertension', 'Fitness Goal_Weight Loss']
target_col = 'diet_Diet_and_lowfat_dairy_products'

X = final_df[feature_cols]
y = final_df[target_col]

# Retrain model
model = RandomForestClassifier()
model.fit(X, y)

# Show interpretable features
importance = pd.Series(model.feature_importances_, index=feature_cols)
print("\nKey Drivers of Diet Choices:")
print(importance.sort_values(ascending=False))

# =================================================================
# 7. Output Sample
# =================================================================
print("\nSample preprocessed data:")
print(final_df.iloc[:2, :5].T)

C:\Users\user\AppData\Local\Temp\ipykernel_3388\2478514432.py:79: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[binary_cols] = df[binary_cols].applymap(standardize_text)


Final features: ['Sex', 'Age', 'Height', 'Weight', 'Hypertension', 'Diabetes', 'BMI', 'Level_Obuse', 'Level_Overweight', 'Level_Underweight', 'Fitness Goal_Weight Loss', 'Fitness Type_Muscular Fitness', 'diet_Diet_aloe_vera_juice', 'diet_Diet_and_apple', 'diet_Diet_and_baru_nuts', 'diet_Diet_and_beetroot_juice', 'diet_Diet_and_bell_peppers', 'diet_Diet_and_black_walnut', 'diet_Diet_and_hemp_seeds', 'diet_Diet_and_iceberg_lettuce', 'diet_Diet_and_jelly_sandwich', 'diet_Diet_and_lettuce', 'diet_Diet_and_lowfat_dairy_products', 'diet_Diet_and_mango_juice', 'diet_Diet_and_mixed_teff', 'diet_Diet_and_nuts', 'diet_Diet_and_watermelon_juice', 'diet_Diet_apple_juice', 'diet_Diet_baru_nuts', 'diet_Diet_beech_nuts'] ...

Key Drivers of Diet Choices:
BMI                         0.531184
Hypertension                0.298081
Age                         0.121141
Fitness Goal_Weight Loss    0.049594
dtype: float64

Sample preprocessed data:
                  0      1
Sex            0.00   0.00
Age   

In [13]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import pickle

# **Step 1: Define user and plan feature columns**
# Assuming 'final_df' is your preprocessed DataFrame
user_cols = ['Sex', 'Age', 'Height', 'Weight', 'Hypertension', 'Diabetes', 'BMI'] + \
            [col for col in final_df.columns if col.startswith(('Level_', 'Fitness Goal_'))]
plan_cols = [col for col in final_df.columns if col.startswith(('exercise_', 'equipment_', 'diet_', 'Fitness Type_'))]

# Ensure all columns are numeric
final_df = final_df.astype(float)

# **Step 2: Incorporate Key Drivers with Feature Weighting**
importance = {
    'BMI': 0.5399,              # High importance
    'Hypertension': 0.2947,     # Moderate importance
    'Age': 0.1200,              # Lower importance
    'Fitness Goal_Weight Loss': 0.0454  # Lowest importance
}

# Scale the key driver features by their importance scores
for col, weight in importance.items():
    if col in user_cols:
        final_df[col] *= weight

# **Step 3: Prepare Positive and Negative Examples**
user_features_pos = final_df[user_cols].values
plan_features_pos = final_df[plan_cols].values
labels_pos = np.ones(len(final_df))  # Positive pairs

# Generate negative examples
num_negatives = 2
neg_user_features = []
neg_plan_features = []
for i in range(len(final_df)):
    for _ in range(num_negatives):
        neg_idx = np.random.randint(0, len(final_df))
        while neg_idx == i:  # Ensure it's not the positive pair
            neg_idx = np.random.randint(0, len(final_df))
        neg_user_features.append(user_features_pos[i])
        neg_plan_features.append(plan_features_pos[neg_idx])

neg_user_features = np.array(neg_user_features)
neg_plan_features = np.array(neg_plan_features)
labels_neg = np.zeros(len(neg_user_features))  # Negative pairs

# Combine positive and negative examples
all_user_features = np.vstack([user_features_pos, neg_user_features])
all_plan_features = np.vstack([plan_features_pos, neg_plan_features])
all_labels = np.hstack([labels_pos, labels_neg])

# **Step 4: Split into Training and Validation Sets**
train_user, val_user, train_plan, val_plan, train_labels, val_labels = train_test_split(
    all_user_features, all_plan_features, all_labels, test_size=0.2, random_state=42
)

# Convert to PyTorch tensors
train_user_tensor = torch.tensor(train_user, dtype=torch.float32)
train_plan_tensor = torch.tensor(train_plan, dtype=torch.float32)
train_label_tensor = torch.tensor(train_labels, dtype=torch.float32)
val_user_tensor = torch.tensor(val_user, dtype=torch.float32)
val_plan_tensor = torch.tensor(val_plan, dtype=torch.float32)
val_label_tensor = torch.tensor(val_labels, dtype=torch.float32)

# **Step 5: Define the CNN-based Plan Tower**
class PlanTowerWithCNN(nn.Module):
    def __init__(self, plan_feature_dim, embedding_dim=64):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveMaxPool1d(1)  # Pools to a fixed size
        )
        self.fc = nn.Linear(64, embedding_dim)
    
    def forward(self, plan_features):
        # Add channel dimension: (batch_size, 1, plan_feature_dim)
        plan_features = plan_features.unsqueeze(1)
        cnn_out = self.cnn(plan_features)  # (batch_size, 64, 1)
        cnn_out = cnn_out.squeeze(2)      # (batch_size, 64)
        plan_emb = self.fc(cnn_out)       # (batch_size, embedding_dim)
        return plan_emb

# **Step 6: Define the Two-Tower Model with CNN in Plan Tower**
class TwoTowerModelWithCNN(nn.Module):
    def __init__(self, user_feature_dim, plan_feature_dim, embedding_dim=64):
        super().__init__()
        # User tower (MLP)
        self.user_encoder = nn.Sequential(
                    nn.Linear(user_feature_dim, 256),
                    nn.ReLU(),
                    nn.Linear(256, 128),  # New layer
                    nn.ReLU(),
                    nn.Linear(128, embedding_dim)
                )
                # Plan tower with an extra layer
        self.plan_encoder = nn.Sequential(
                    nn.Linear(plan_feature_dim, 256),
                    nn.ReLU(),
                    nn.Linear(256, 128),  # New layer
                    nn.ReLU(),
                    nn.Linear(128, embedding_dim)
                )
        # Plan tower (CNN)
        self.plan_encoder = PlanTowerWithCNN(plan_feature_dim, embedding_dim)
    
    def forward(self, user_features, plan_features):
        user_emb = self.user_encoder(user_features)
        plan_emb = self.plan_encoder(plan_features)
        score = (user_emb * plan_emb).sum(dim=1)
        return torch.sigmoid(score)  # Output probabilities

# Initialize the model
model = TwoTowerModelWithCNN(user_feature_dim=len(user_cols), plan_feature_dim=len(plan_cols))

# **Step 7: Train the Model and Track Performance Metrics**
train_dataset = TensorDataset(train_user_tensor, train_plan_tensor, train_label_tensor)
val_dataset = TensorDataset(val_user_tensor, val_plan_tensor, val_label_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

num_epochs = 50
for epoch in range(num_epochs):
    # Training phase
    model.train()
    total_loss = 0
    for user_batch, plan_batch, label_batch in train_loader:
        optimizer.zero_grad()
        scores = model(user_batch, plan_batch)
        loss = criterion(scores, label_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Validation phase
    model.eval()
    val_scores = []
    val_true_labels = []
    with torch.no_grad():
        for user_batch, plan_batch, label_batch in val_loader:
            scores = model(user_batch, plan_batch)
            val_scores.extend(scores.numpy())
            val_true_labels.extend(label_batch.numpy())

    val_scores = np.array(val_scores)
    val_true_labels = np.array(val_true_labels)
    val_predictions = (val_scores > 0.5).astype(int)
    precision = precision_score(val_true_labels, val_predictions)
    recall = recall_score(val_true_labels, val_predictions)
    f1 = f1_score(val_true_labels, val_predictions)
    auc = roc_auc_score(val_true_labels, val_scores)

    print(f"Epoch {epoch+1}, Train Loss: {total_loss / len(train_loader):.4f}, "
          f"Val Precision: {precision:.4f}, Val Recall: {recall:.4f}, Val F1: {f1:.4f}, Val AUC: {auc:.4f}")

# **Step 8: Save the Model and Additional Data**
torch.save(model.state_dict(), 'recommendation_model_with_cnn.pth')

with open('model_data.pkl', 'wb') as f:
    pickle.dump({
        'user_cols': user_cols,
        'plan_cols': plan_cols,
        'importance': importance
    }, f)

print("Model and data saved successfully!")

Epoch 1, Train Loss: 0.5818, Val Precision: 0.5307, Val Recall: 0.4299, Val F1: 0.4750, Val AUC: 0.7730
Epoch 2, Train Loss: 0.4731, Val Precision: 0.5337, Val Recall: 0.8429, Val F1: 0.6536, Val AUC: 0.8091
Epoch 3, Train Loss: 0.4351, Val Precision: 0.6620, Val Recall: 0.5005, Val F1: 0.5701, Val AUC: 0.8642
Epoch 4, Train Loss: 0.4032, Val Precision: 0.6714, Val Recall: 0.8280, Val F1: 0.7415, Val AUC: 0.8743
Epoch 5, Train Loss: 0.3845, Val Precision: 0.6763, Val Recall: 0.9044, Val F1: 0.7739, Val AUC: 0.8944
Epoch 6, Train Loss: 0.3704, Val Precision: 0.6923, Val Recall: 0.6411, Val F1: 0.6657, Val AUC: 0.8817
Epoch 7, Train Loss: 0.3620, Val Precision: 0.6993, Val Recall: 0.8503, Val F1: 0.7674, Val AUC: 0.8936
Epoch 8, Train Loss: 0.3549, Val Precision: 0.6992, Val Recall: 0.9081, Val F1: 0.7901, Val AUC: 0.9025
Epoch 9, Train Loss: 0.3465, Val Precision: 0.7003, Val Recall: 0.9594, Val F1: 0.8096, Val AUC: 0.9170
Epoch 10, Train Loss: 0.3256, Val Precision: 0.7668, Val Recall: